In [1]:
"""
Polynomial multiplication
"""
def cl_mul(x: int,y: int) -> int:
    """Bitwise carry-less multiplication on integers, or similarly polynomial multiplication"""
    z: int = 0
    i: int = 0
    while (y>>i) > 0:
        if y & (1<<i):
            z ^= x<<i
        i += 1
    return z

In [11]:
# Example: (x^2 + x + 1)(0x^2 + 1x + 0) = 0x^4 + 1x^3 + 1x^2 + 1x^1 + 0
print(f"{cl_mul(0b111,0b010):05b}")

01110


In [3]:
"""
Polynomial division
"""
def cl_div(dividend: int, divisor: int) -> tuple[int, int]:
    """
    Carry-less long division of two integers.
    Returns the [quotient, remainder] of dividend/divisor (using XOR as subtraction).
    """
    dl1: int = dividend.bit_length()
    dl2: int = divisor.bit_length()
    quotient: int = 0
    if dl1 < dl2:
        return (quotient, dividend)
    # Align the most significant bit of divisor with that of dividend, 
    # so remainder will be at most 1 bit less than divisor (9-bit divisor, 8-bit or less remainder)
    for i in range(dl1 - dl2, -1, -1):
        if dividend & (1 << (i + dl2 - 1)):
            dividend ^= divisor << i
            quotient ^= 1 << i
    return (quotient, dividend)

In [4]:
dividend = 0b1011
divisor = 0b111
result = cl_div(dividend, divisor)
print(f"(quotient={result[0]:b},remainder={result[1]:b})")
print(f"dividend=divisor x quotient + remainder: {dividend=:b}={cl_mul(divisor, result[0])^result[1]:b}")

(quotient=11,remainder=10)
dividend=divisor x quotient + remainder: dividend=1011=1011


In [5]:
"""
Galois field multiplication
"""
def gf_mult_noLUT(x: int, y: int, prim: int = 0) -> int:
    """
    Multiplication in Galois Fields without using a precomputed look-up table.
    It performs carry-less multiplication and then, if prim > 0, reduces modulo the
    prime polynomial `prim`.

    :param x: First operand (as an integer representing a GF(2) polynomial)
    :param y: Second operand
    :param prim: Primitive polynomial for modular reduction (default 0 means no reduction)
    :return: The product in GF(2^n) (with modular reduction if prim > 0)
    """

    def cl_mult(x: int, y: int) -> int:
        """Carry-less multiplication of two integers (treating them as GF(2) polynomials)."""
        z: int = 0
        i: int = 0
        while (y >> i) > 0:
            if y & (1 << i):
                z ^= x << i
            i += 1
        return z

    def bit_length(n: int) -> int:
        """
        Compute the position of the most significant 1-bit in an integer.
        Equivalent to int.bit_length() built-in function.
        """
        bits: int = 0
        while n >> bits:
            bits += 1
        return bits

    def cl_div(dividend: int, divisor: int) -> int:
        """
        Carry-less long division of two integers.
        Returns the remainder of dividend/divisor (using XOR as subtraction).
        Equivalent to polynomial remainder of polynomial division.
        """
        dl1: int = bit_length(dividend)
        dl2: int = bit_length(divisor)
        if dl1 < dl2:
            return dividend
        # Align the most significant bit of divisor with that of dividend, 
        # so remainder will be at most 1 bit less than divisor (9-bit divisor, 8-bit or less remainder)
        for i in range(dl1 - dl2, -1, -1):
            if dividend & (1 << (i + dl2 - 1)):
                dividend ^= divisor << i
        return dividend

    # Perform carry-less multiplication
    result: int = cl_mult(x, y)

    # If a primitive polynomial is provided, reduce the product modulo prim
    if prim > 0:
        result = cl_div(result, prim)

    return result